<a href="https://colab.research.google.com/github/aymnblh/AI/blob/main/laptop_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot  as plt
import numpy as np
import string
import nltk as nk
import nltk.corpus
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import words
from sklearn.model_selection import train_test_split
import nltk

In [ ]:
data=pd.read_csv("/content/data_1_train.csv",sep = ',')
data.head()

,example_id,text,aspect_term,term_location,class
0,2333_1,Obviously one of the most important features o...,human interface,69--84,0
1,1805_1,Good for every day computing and web browsing.,every day computing,9--28,1
2,2782_2,while the keyboard itself is alright[comma] th...,mouse command buttons,115--136,-1
3,1385_0,Again[comma] the same problem[comma] the right...,right speaker,29--42,-1
4,1423_0,My problem was with DELL Customer Service.,DELL Customer Service,20--41,-1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2203 entries, 0 to 2202
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   example_id      2203 non-null   object
 1    text           2203 non-null   object
 2    aspect_term    2203 non-null   object
 3    term_location  2203 non-null   object
 4    class          2203 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 86.2+ KB


In [ ]:
data.groupby([' class'])[' text'].count()

 class
-1    828
 0    436
 1    939
Name:  text, dtype: int64

In [ ]:
data["phrase_len"]= data[" text"].str.len() 
data.head(100)

,example_id,text,aspect_term,term_location,class,phrase_len
0,2333_1,Obviously one of the most important features o...,human interface,69--84,0,85
1,1805_1,Good for every day computing and web browsing.,every day computing,9--28,1,46
2,2782_2,while the keyboard itself is alright[comma] th...,mouse command buttons,115--136,-1,143
3,1385_0,Again[comma] the same problem[comma] the right...,right speaker,29--42,-1,68
4,1423_0,My problem was with DELL Customer Service.,DELL Customer Service,20--41,-1,42
...,...,...,...,...,...,...
95,899_0,The computer itself was fast[comma] ran smooth...,ran,30--33,1,76
96,908_0,Also[comma] my sister got the exact same lapto...,screen,93--99,-1,149
97,1812_0,The difference is it's a whole lot of fun usin...,Apple navigation,83--99,0,158
98,462_0,Unable to boot up this brand new laptop.,boot up,10--17,-1,40


In [ ]:
data.groupby([' class'])['phrase_len'].mean()

 class
-1    112.927536
 0    125.720183
 1    117.336528
Name: phrase_len, dtype: float64

In [ ]:
data.loc[data['phrase_len']<3,' text']='Null'

In [ ]:
data.pop("phrase_len")

0        85
1        46
2       143
3        68
4        42
       ... 
2198    117
2199    154
2200     78
2201     33
2202    155
Name: phrase_len, Length: 2203, dtype: int64

In [ ]:
data['Review']=data[' text']
data.pop(" text")
data['Liked']=data[' class']
data.pop(" class")

0       0
1       1
2      -1
3      -1
4      -1
       ..
2198    0
2199    1
2200    1
2201    1
2202    0
Name:  class, Length: 2203, dtype: int64

In [ ]:
import string
data['Review'] = data['Review'].apply(lambda x: x.replace('[comma]',',').lower())
string.punctuation
def remove_punctuation(txt):
    txt_nopunt="".join([c for c in txt if c not in string.punctuation])
    return txt_nopunt
data ['Review'] = data ['Review'].apply(lambda x: remove_punctuation(x))
data ['Review'] = data ['Review']. str.replace ('br' , '')
data ['Review'] = data ['Review']. str.lower () 

data.head()

,example_id,aspect_term,term_location,Review,Liked
0,2333_1,human interface,69--84,obviously one of the most important features o...,0
1,1805_1,every day computing,9--28,good for every day computing and web owsing,1
2,2782_2,mouse command buttons,115--136,while the keyboard itself is alright the plate...,-1
3,1385_0,right speaker,29--42,again the same problem the right speaker did n...,-1
4,1423_0,DELL Customer Service,20--41,my problem was with dell customer service,-1


In [ ]:
import spacy
lst = data[" aspect_term"].tolist()
nlp = spacy.load('en_core_web_sm', parser=False)
def pos(text):
    doc = nlp(text)
    aspects = [token.text for token in doc if token.pos_ == 'NOUN' ]
    for element in aspects:
     if element not in lst:
       aspects.remove(element)
    return aspects
data["Aspects"] = data["Review"].apply(pos)
data.head(15)

,example_id,aspect_term,term_location,Review,Liked,Aspects
0,2333_1,human interface,69--84,obviously one of the most important features o...,0,"[features, interface]"
1,1805_1,every day computing,9--28,good for every day computing and web owsing,1,"[computing, owsing]"
2,2782_2,mouse command buttons,115--136,while the keyboard itself is alright the plate...,-1,"[keyboard, plate, sound, mouse, buttons]"
3,1385_0,right speaker,29--42,again the same problem the right speaker did n...,-1,[speaker]
4,1423_0,DELL Customer Service,20--41,my problem was with dell customer service,-1,"[customer, service]"
5,104_0,Windows Vista,53--66,is this partially due to the fact that it is r...,0,[]
6,398_0,hard drive,117--127,accordingly i have decided to never purchase a...,-1,"[year, drive]"
7,2766_1,specifications,96--110,i did think it had a camera because that was o...,0,"[camera, specifications]"
8,2808_5,setup,138--143,cords coming out the right for power plus cord...,-1,"[cords, power, cords, network, connection, set..."
9,1727_0,Safari internet browser,30--53,the most recent being that my safari internet ...,-1,"[internet, firefox]"


In [ ]:
nltk.download('punkt')
data['Review'] = data.apply(lambda row: nltk.word_tokenize(row['Review']), axis=1)
data[' aspect_term'] = data[' aspect_term'].apply(lambda x: x.lower())
data[' aspect_term'] = data.apply(lambda row: nltk.word_tokenize(row[' aspect_term']), axis=1)
data[' aspect_term'] = data[' aspect_term'].apply(lambda x:" ".join(x))
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,example_id,aspect_term,term_location,Review,Liked,Aspects
0,2333_1,human interface,69--84,"[obviously, one, of, the, most, important, fea...",0,"[features, interface]"
1,1805_1,every day computing,9--28,"[good, for, every, day, computing, and, web, o...",1,"[computing, owsing]"
2,2782_2,mouse command buttons,115--136,"[while, the, keyboard, itself, is, alright, th...",-1,"[keyboard, plate, sound, mouse, buttons]"
3,1385_0,right speaker,29--42,"[again, the, same, problem, the, right, speake...",-1,[speaker]
4,1423_0,dell customer service,20--41,"[my, problem, was, with, dell, customer, service]",-1,"[customer, service]"


In [ ]:
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [ ]:
l=["ain", "aren", "aren't", "couldn", "couldn't", "didn", "didn't", "doesn", "doesn't", "hadn", "hadn't", 'hasn', "hasn't", "haven", "haven't", "isn", "isn't","mightn", "mightn't","mustn", "mustn't", "needn", "needn't", "shan", "shan't", "shouldn", "shouldn't", "wasn", "wasn't", "weren", "weren't", "won't", "wouldn", "wouldn't","no","nor","not","don", "don't"]
pd.set_option('display.max_colwidth', 0)
stop = stopwords.words('english')
for element in l:
    if element in stop:
        stop.remove(element)  
data['Review'] = data['Review'].apply(lambda x:[c for c in x if c not in stop])
data.head()

,example_id,aspect_term,term_location,Review,Liked,Aspects
0,2333_1,human interface,69--84,"[obviously, one, important, features, computer, human, interface]",0,"[features, interface]"
1,1805_1,every day computing,9--28,"[good, every, day, computing, web, owsing]",1,"[computing, owsing]"
2,2782_2,mouse command buttons,115--136,"[keyboard, alright, plate, around, cheap, plastic, makes, hollow, sound, using, mouse, command, buttons]",-1,"[keyboard, plate, sound, mouse, buttons]"
3,1385_0,right speaker,29--42,"[problem, right, speaker, not, work]",-1,[speaker]
4,1423_0,dell customer service,20--41,"[problem, dell, customer, service]",-1,"[customer, service]"


In [ ]:
data = data.rename(columns = {' aspect_term': 'aspect_term'}, inplace = False)

In [ ]:
stemmer = nltk.stem.SnowballStemmer('english')
data.Review = data.apply(lambda row: [stemmer.stem(item) for item in row.Review], axis=1)
data.head()

,example_id,aspect_term,term_location,Review,Liked,Aspects
0,2333_1,human interface,69--84,"[obvious, one, import, featur, comput, human, interfac]",0,"[features, interface]"
1,1805_1,every day computing,9--28,"[good, everi, day, comput, web, ows]",1,"[computing, owsing]"
2,2782_2,mouse command buttons,115--136,"[keyboard, alright, plate, around, cheap, plastic, make, hollow, sound, use, mous, command, button]",-1,"[keyboard, plate, sound, mouse, buttons]"
3,1385_0,right speaker,29--42,"[problem, right, speaker, not, work]",-1,[speaker]
4,1423_0,dell customer service,20--41,"[problem, dell, custom, servic]",-1,"[customer, service]"


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
data['Review']=data['Review'].astype(str)
v = TfidfVectorizer()
x = v.fit_transform(data['Review'])
print('Tfidf:',x.shape)

Tfidf: (2203, 2560)


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#tokenize and tag the card text
card_docs = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(data.Review)]


In [ ]:
model = Doc2Vec(vector_size=64, min_count=2, epochs = 20)

In [ ]:
#instantiate model
model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)
#build vocab
model.build_vocab(card_docs)
#train model
model.train(card_docs, total_examples=model.corpus_count
            , epochs=model.epochs)

In [ ]:
#generate vectors
doc2vec = [model.infer_vector((data['Review'][i].split(' '))) 
            for i in range(0,len(data['Review']))]


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#tokenize and tag the card text
card_docs3 = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(data. aspect_term)]

In [ ]:
#instantiate model
model3 = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)
#build vocab
model3.build_vocab(card_docs3)
#train model
model3.train(card_docs3, total_examples=model.corpus_count
            , epochs=model.epochs)

In [ ]:
#generate vectors
aspect2vec = [model3.infer_vector((data['aspect_term'][i].split(' '))) 
            for i in range(0,len(data['aspect_term']))]

In [ ]:
#Create a list of lists
dtv3= np.array(aspect2vec).tolist()
#set list to dataframe column
data['aspect2vec'] = dtv3

In [ ]:
import numpy as np
#Create a list of lists
dtv= np.array(doc2vec).tolist()
#set list to dataframe column
data['doc2vec'] = dtv
ln = len(data['doc2vec'])

In [ ]:
x1 = list(data['doc2vec'])
x2 = list(data['aspect2vec'])
x = list(zip(x1,x2))
y = data['Liked']
x = np.array(x)
x = x.reshape(x.shape[0], -1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import tree   
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
#splitting data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=27)  #splitting data with test size of 30%

#applying tree algorithm
tree = tree.DecisionTreeClassifier()  
tree.fit(x_train, y_train)   #fitting our model
y_pred=tree.predict(x_test)   # evaluating our model
print("score:{}".format(accuracy_score(y_test, y_pred)))

score:0.540090771558245


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.54      0.52      0.53       246
           0       0.36      0.42      0.39       123
           1       0.63      0.60      0.62       292

    accuracy                           0.54       661
   macro avg       0.51      0.52      0.51       661
weighted avg       0.55      0.54      0.54       661



In [ ]:
from sklearn.ensemble import RandomForestClassifier #Importing Random Forest Classifier
from sklearn import metrics  # Importing metrics to test accuracy
clf=RandomForestClassifier(n_estimators=10)  #Creating a random forest with 100 decision trees
clf.fit(x_train, y_train)  #Training our model
y_pred=clf.predict(x_test)  #testing our model
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))  #Measuring the accuracy of our model

Accuracy: 0.6338880484114977


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.61      0.72      0.66       246
           0       0.45      0.37      0.40       123
           1       0.73      0.67      0.70       292

    accuracy                           0.63       661
   macro avg       0.60      0.59      0.59       661
weighted avg       0.63      0.63      0.63       661



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(
             x, y, test_size = 0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

y_pred=knn.predict(X_test)

print('Accuracy ', accuracy_score(y_pred,y_test))

Accuracy  0.6417233560090703


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.67      0.64      0.65       184
           0       0.46      0.48      0.47        88
           1       0.71      0.73      0.72       169

    accuracy                           0.64       441
   macro avg       0.61      0.62      0.61       441
weighted avg       0.64      0.64      0.64       441



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
#splitting data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=27)  #splitting data with test size of 30%
logreg = LogisticRegression()   #build our logistic model
logreg.fit(x_train, y_train)  #fitting training data
y_pred  = logreg.predict(x_test)    #testing model’s performance
print("Accuracy={:.2f}".format(logreg.score(x_test, y_test)))
print("Accuracy train data={:.2f}".format(logreg.score(x_train, y_train)))

Accuracy=0.65
Accuracy train data=0.64


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.63      0.67      0.65       246
           0       0.51      0.20      0.28       123
           1       0.68      0.83      0.75       292

    accuracy                           0.65       661
   macro avg       0.61      0.56      0.56       661
weighted avg       0.63      0.65      0.62       661



In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x, y);
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))  #Measuring the accuracy of our model

Accuracy: 0.6505295007564297


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.63      0.67      0.65       246
           0       0.51      0.20      0.28       123
           1       0.68      0.83      0.75       292

    accuracy                           0.65       661
   macro avg       0.61      0.56      0.56       661
weighted avg       0.63      0.65      0.62       661



In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
x1 = list(data['doc2vec'])
x2 = list(data['aspect2vec'])
x = list(zip(x1,x2))
y = data['Liked']
x = np.array(x)
x = x.reshape(x.shape[0], -1)
y=tf.keras.utils.to_categorical(y, 3)
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25,random_state=35)
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu')
])
model.add(Dense(128, activation='relu'))
model.add(Dense(3, input_dim=128, activation='softmax'))



model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
y_pred = model.predict(x_test)

y_pred = np.argmax(y_pred, axis=1)
model.fit(x_train, y_train, epochs=500)

Epoch 1/1000
52/52 [==============================] - 1s 2ms/step - loss: 0.9891 - accuracy: 0.5363
Epoch 2/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.8844 - accuracy: 0.6144
Epoch 3/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.8479 - accuracy: 0.6271
Epoch 4/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.8111 - accuracy: 0.6501
Epoch 5/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.7788 - accuracy: 0.6671
Epoch 6/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.7554 - accuracy: 0.6774
Epoch 7/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.7296 - accuracy: 0.6804
Epoch 8/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.7105 - accuracy: 0.6949
Epoch 9/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.6879 - accuracy: 0.7119
Epoch 10/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.6731 - accuracy: 0.7149

In [ ]:
model.evaluate(x_test,y_test)

18/18 [==============================] - 0s 1ms/step - loss: 4.4335 - accuracy: 0.6243


[4.433513641357422, 0.6243194341659546]

In [ ]:
y_pred.shape

(551,)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.21      0.40      0.27       111
           1       0.29      0.31      0.30       240
           2       0.35      0.15      0.21       200

    accuracy                           0.27       551
   macro avg       0.28      0.29      0.26       551
weighted avg       0.30      0.27      0.26       551



In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.layers import Dropout
x1 = list(data['doc2vec'])
x2 = list(data['aspect2vec'])
x = list(zip(x1,x2))
y = data['Liked']
x = np.array(x)
x = x.reshape(x.shape[0], -1)
y=tf.keras.utils.to_categorical(y, 3)
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25,random_state=35)
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu')
])
model.add(Dense(64, input_dim=128, activation='relu'))
model.add(Dense(64, input_dim=64, activation='relu'))
model.add(Dense(3, input_dim=64, activation='softmax'))
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
y_pred = model.predict(x_test)

y_pred = np.argmax(y_pred, axis=1)
model.fit(x_train, y_train, epochs=500)

Epoch 1/1000
52/52 [==============================] - 1s 2ms/step - loss: 1.0023 - accuracy: 0.5321
Epoch 2/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.8931 - accuracy: 0.6199
Epoch 3/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.8470 - accuracy: 0.6314
Epoch 4/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.8182 - accuracy: 0.6477
Epoch 5/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.7802 - accuracy: 0.6640
Epoch 6/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.7437 - accuracy: 0.6858
Epoch 7/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.7212 - accuracy: 0.6998
Epoch 8/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.6885 - accuracy: 0.7119
Epoch 9/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.6732 - accuracy: 0.7197
Epoch 10/1000
52/52 [==============================] - 0s 2ms/step - loss: 0.6461 - accuracy: 0.7361

In [ ]:
model.evaluate(x_test,y_test)

18/18 [==============================] - 0s 1ms/step - loss: 6.9047 - accuracy: 0.6279


[6.904654026031494, 0.6279491782188416]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.22      0.17      0.19       111
           1       0.00      0.00      0.00       240
           2       0.38      0.87      0.52       200

    accuracy                           0.35       551
   macro avg       0.20      0.35      0.24       551
weighted avg       0.18      0.35      0.23       551



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.layers import Dropout
x1 = list(data['doc2vec'])
x2 = list(data['aspect2vec'])
x = list(zip(x1,x2))
y = data['Liked']
x = np.array(x)
x = x.reshape(x.shape[0], -1)
y=tf.keras.utils.to_categorical(y, 3)
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25,random_state=35)
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu')
])
model.add(Dense(64, input_dim=128, activation='relu'))
model.add(Dense(64, input_dim=128, activation='relu'))
model.add(Dense(64, input_dim=64, activation='relu'))
model.add(Dense(3, input_dim=64, activation='softmax'))
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
y_pred = model.predict(x_test)

y_pred = np.argmax(y_pred, axis=1)
model.fit(x_train, y_train, epochs=500)

Streaming output truncated to the last 5000 lines.
52/52 [==============================] - 0s 4ms/step - loss: 4.3000e-07 - accuracy: 1.0000
Epoch 2502/5000
52/52 [==============================] - 0s 4ms/step - loss: 4.2336e-07 - accuracy: 1.0000
Epoch 2503/5000
52/52 [==============================] - 0s 4ms/step - loss: 4.0431e-07 - accuracy: 1.0000
Epoch 2504/5000
52/52 [==============================] - 0s 4ms/step - loss: 3.9926e-07 - accuracy: 1.0000
Epoch 2505/5000
52/52 [==============================] - 0s 4ms/step - loss: 3.9118e-07 - accuracy: 1.0000
Epoch 2506/5000
52/52 [==============================] - 0s 4ms/step - loss: 3.7408e-07 - accuracy: 1.0000
Epoch 2507/5000
52/52 [==============================] - 0s 4ms/step - loss: 3.6145e-07 - accuracy: 1.0000
Epoch 2508/5000
52/52 [==============================] - 0s 4ms/step - loss: 3.6729e-07 - accuracy: 1.0000
Epoch 2509/5000
52/52 [==============================] - 0s 4ms/step - loss: 3.4767e-07 - accuracy: 1.0000
Ep

In [ ]:
model.evaluate(x_test,y_test)

18/18 [==============================] - 0s 2ms/step - loss: 15.0406 - accuracy: 0.6243


[15.040648460388184, 0.6243194341659546]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.22      0.32      0.26       111
           1       1.00      0.01      0.02       240
           2       0.35      0.68      0.46       200

    accuracy                           0.31       551
   macro avg       0.52      0.33      0.25       551
weighted avg       0.61      0.31      0.23       551

